In [1]:
import numpy as np
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
from matplotlib import style
import matplotlib.ticker as ticker
import seaborn as sns

In [3]:
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, make_scorer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.inspection import permutation_importance
import multiprocessing

In [4]:
from mlxtend.classifier import EnsembleVoteClassifier
from sklearn.ensemble import BaggingClassifier

In [5]:
from xgboost import XGBClassifier

In [6]:
labels = pd.read_csv('../../csv/train_labels.csv')
labels.head()

,building_id,damage_grade
0,802906,3
1,28830,2
2,94947,3
3,590882,2
4,201944,3


In [7]:
values = pd.read_csv('../../csv/train_values.csv')
values.T

,0,1,2,3,4,5,6,7,8,9,...,260591,260592,260593,260594,260595,260596,260597,260598,260599,260600
building_id,802906,28830,94947,590882,201944,333020,728451,475515,441126,989500,...,560805,207683,226421,159555,827012,688636,669485,602512,151409,747594
geo_level_1_id,6,8,21,22,11,8,9,20,0,26,...,20,10,8,27,8,25,17,17,26,21
geo_level_2_id,487,900,363,418,131,558,475,323,757,886,...,368,1382,767,181,268,1335,715,51,39,9
geo_level_3_id,12198,2812,8973,10694,1488,6089,12066,12236,7219,994,...,5980,1903,8613,1537,4718,1621,2060,8163,1851,9101
count_floors_pre_eq,2,2,2,2,3,2,2,2,2,1,...,1,2,2,6,2,1,2,3,2,3
age,30,10,10,10,30,10,25,0,15,0,...,25,25,5,0,20,55,0,55,10,10
area_percentage,6,8,5,6,8,9,3,8,8,13,...,5,5,13,13,8,6,6,6,14,7
height_percentage,5,7,5,5,9,5,4,6,6,4,...,3,5,5,12,5,3,5,7,6,6
land_surface_condition,t,o,t,t,t,t,n,t,t,t,...,n,t,t,t,t,n,t,t,t,n
foundation_type,r,r,r,r,r,r,r,w,r,i,...,r,r,r,r,r,r,r,r,r,r


In [8]:
to_be_categorized = ["land_surface_condition", "foundation_type", "roof_type",\
                     "position", "ground_floor_type", "other_floor_type",\
                     "plan_configuration", "legal_ownership_status"]
for row in to_be_categorized:
    values[row] = values[row].astype("category")
values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260601 entries, 0 to 260600
Data columns (total 39 columns):
 #   Column                                  Non-Null Count   Dtype   
---  ------                                  --------------   -----   
 0   building_id                             260601 non-null  int64   
 1   geo_level_1_id                          260601 non-null  int64   
 2   geo_level_2_id                          260601 non-null  int64   
 3   geo_level_3_id                          260601 non-null  int64   
 4   count_floors_pre_eq                     260601 non-null  int64   
 5   age                                     260601 non-null  int64   
 6   area_percentage                         260601 non-null  int64   
 7   height_percentage                       260601 non-null  int64   
 8   land_surface_condition                  260601 non-null  category
 9   foundation_type                         260601 non-null  category
 10  roof_type                       

In [9]:
datatypes = dict(values.dtypes)
for row in values.columns:
    if datatypes[row] != "int64" and datatypes[row] != "int32" and \
       datatypes[row] != "int16" and datatypes[row] != "int8":
        continue
    if values[row].nlargest(1).item() > 32767 and values[row].nlargest(1).item() < 2**31:
        values[row] = values[row].astype(np.int32)
    elif values[row].nlargest(1).item() > 127:
        values[row] = values[row].astype(np.int16)
    else:
        values[row] = values[row].astype(np.int8)

In [10]:
labels["building_id"] = labels["building_id"].astype(np.int32)
labels["damage_grade"] = labels["damage_grade"].astype(np.int8)
labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260601 entries, 0 to 260600
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype
---  ------        --------------   -----
 0   building_id   260601 non-null  int32
 1   damage_grade  260601 non-null  int8 
dtypes: int32(1), int8(1)
memory usage: 1.2 MB


In [11]:
values['age_is_leq_than_100'] = (values['age'] <= 100).astype(np.int8)
# values['age_is_betw_100_and_200'] = ((values['age'] > 100) & (values['age'] <= 200)).astype(np.int8)
values['age_is_greater_than_200'] = (values['age'] > 200).astype(np.int8)
values[values['age'] >= 100]

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,age_is_leq_than_100,age_is_greater_than_200
216,612661,11,883,2822,5,100,12,11,t,r,...,0,0,0,0,0,0,0,0,1,0
324,737045,21,477,4348,3,190,6,7,t,r,...,0,0,0,0,0,0,0,0,0,0
386,435557,14,1120,3789,3,995,12,5,t,r,...,0,0,0,0,0,0,0,0,0,1
419,370360,6,1253,5667,4,105,11,9,t,r,...,0,0,0,0,0,0,0,0,0,0
472,393373,10,310,3637,2,100,4,4,t,r,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260093,34321,17,1393,5939,2,995,8,4,t,r,...,0,0,0,0,0,0,0,0,0,1
260469,1038559,3,1387,9059,5,105,9,11,t,r,...,0,0,0,0,0,0,0,0,0,0
260541,798100,13,1365,11849,1,100,6,3,t,r,...,0,0,0,0,0,0,0,0,1,0
260542,156434,8,696,7863,2,995,8,8,t,r,...,0,0,0,0,0,0,0,0,0,1


# Feature Engineering para XGBoost

In [12]:
important_values = values\
                .merge(labels, on="building_id")
important_values.drop(columns=["building_id"], inplace = True)
important_values["geo_level_1_id"] = important_values["geo_level_1_id"].astype("category")
important_values

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,age_is_leq_than_100,age_is_greater_than_200,damage_grade
0,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,1,0,3
1,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,1,0,2
2,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,1,0,3
3,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,1,0,2
4,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,1,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260596,25,1335,1621,1,55,6,3,n,r,n,...,0,0,0,0,0,0,0,1,0,2
260597,17,715,2060,2,0,6,5,t,r,n,...,0,0,0,0,0,0,0,1,0,3
260598,17,51,8163,3,55,6,7,t,r,q,...,0,0,0,0,0,0,0,1,0,3
260599,26,39,1851,2,10,14,6,t,r,x,...,0,0,0,0,0,0,0,1,0,2


In [13]:

X_train, X_test, y_train, y_test = train_test_split(important_values.drop(columns = 'damage_grade'),
                                                    important_values['damage_grade'], test_size = 0.075, random_state = 123)

In [14]:
#OneHotEncoding
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res) 

features_to_encode = ["geo_level_1_id", "land_surface_condition", "foundation_type", "roof_type",\
                     "position", "ground_floor_type", "other_floor_type",\
                     "plan_configuration", "legal_ownership_status"]
for feature in features_to_encode:
    X_train = encode_and_bind(X_train, feature)
    X_test = encode_and_bind(X_test, feature)

In [15]:
X_train

,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,...,plan_configuration_m,plan_configuration_n,plan_configuration_o,plan_configuration_q,plan_configuration_s,plan_configuration_u,legal_ownership_status_a,legal_ownership_status_r,legal_ownership_status_v,legal_ownership_status_w
55400,1425,12125,2,30,17,5,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
83245,39,11440,1,15,8,3,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
231531,175,5357,4,40,12,9,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
26746,566,2239,3,25,7,7,1,1,0,0,...,0,0,0,0,0,0,0,0,1,0
225198,70,9429,2,10,5,4,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192476,217,10644,1,25,4,6,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
17730,600,4813,2,20,13,8,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
28030,463,4692,2,10,9,4,1,1,0,0,...,0,0,0,0,0,0,0,0,1,0
15725,600,157,2,50,5,8,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [16]:
xgb_model_1 = XGBClassifier(n_estimators = 350,
                          subsample = 0.885,
                          booster = 'gbtree',
                          gamma = 1,
                          learning_rate = 0.45,
                          label_encoder = False,
                          verbosity = 2)

bc_xgb_model_1 = BaggingClassifier(base_estimator = xgb_model_1,
                        n_estimators = 5,
                        oob_score = True,
                        bootstrap_features = True,
                        random_state = 0,
                        verbose = 3)


bc_xgb_model_1.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/home/joaquinfontela/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Building estimator 1 of 5 for this parallel run (total 5)...
[21:09:23] WARNING: ../src/learner.cc:573: 
Parameters: { "label_encoder" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[21:09:23] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 120 extra nodes, 4 pruned nodes, max_depth=6
[21:09:23] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 112 extra nodes, 6 pruned nodes, max_depth=6
[21:09:24] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 122 extra nodes, 2 pruned nodes, max_depth=6
[21:09:24] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:09:24

[21:09:37] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 90 extra nodes, 16 pruned nodes, max_depth=6
[21:09:37] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 102 extra nodes, 4 pruned nodes, max_depth=6
[21:09:37] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 98 extra nodes, 8 pruned nodes, max_depth=6
[21:09:37] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 72 extra nodes, 14 pruned nodes, max_depth=6
[21:09:38] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 64 extra nodes, 14 pruned nodes, max_depth=6
[21:09:38] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 124 extra nodes, 2 pruned nodes, max_depth=6
[21:09:38] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 110 extra nodes, 4 pruned nodes, max_depth=6
[21:09:38] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 114 extra nodes, 6 pruned nodes, max_depth=6
[21:09:38] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 90 extra nodes, 14 p

[21:09:53] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 104 extra nodes, 4 pruned nodes, max_depth=6
[21:09:53] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 104 extra nodes, 8 pruned nodes, max_depth=6
[21:09:53] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 108 extra nodes, 6 pruned nodes, max_depth=6
[21:09:53] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 108 extra nodes, 8 pruned nodes, max_depth=6
[21:09:53] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 68 extra nodes, 8 pruned nodes, max_depth=6
[21:09:53] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 118 extra nodes, 4 pruned nodes, max_depth=6
[21:09:54] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 78 extra nodes, 10 pruned nodes, max_depth=6
[21:09:54] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 82 extra nodes, 12 pruned nodes, max_depth=6
[21:09:54] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 74 extra nodes, 8 pr

[21:10:07] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 82 extra nodes, 16 pruned nodes, max_depth=6
[21:10:07] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 90 extra nodes, 8 pruned nodes, max_depth=6
[21:10:07] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 78 extra nodes, 14 pruned nodes, max_depth=6
[21:10:07] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 80 extra nodes, 4 pruned nodes, max_depth=6
[21:10:08] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 104 extra nodes, 8 pruned nodes, max_depth=6
[21:10:08] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 118 extra nodes, 2 pruned nodes, max_depth=6
[21:10:08] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 94 extra nodes, 8 pruned nodes, max_depth=6
[21:10:08] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 76 extra nodes, 14 pruned nodes, max_depth=6
[21:10:08] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 94 extra nodes, 12 pru

...

[21:27:40] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 78 extra nodes, 12 pruned nodes, max_depth=6
[21:27:40] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 78 extra nodes, 12 pruned nodes, max_depth=6
[21:27:41] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 66 extra nodes, 22 pruned nodes, max_depth=6
[21:27:41] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 40 extra nodes, 36 pruned nodes, max_depth=6
[21:27:41] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 52 extra nodes, 14 pruned nodes, max_depth=6
[21:27:41] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 56 extra nodes, 8 pruned nodes, max_depth=6
[21:27:42] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 38 extra nodes, 12 pruned nodes, max_depth=6
[21:27:42] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 88 extra nodes, 22 pruned nodes, max_depth=6
[21:27:42] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 86 extra nodes, 24 p

[21:27:58] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 44 extra nodes, 54 pruned nodes, max_depth=6
[21:27:58] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 108 extra nodes, 10 pruned nodes, max_depth=6
[21:27:58] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 92 extra nodes, 26 pruned nodes, max_depth=6
[21:27:58] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 86 extra nodes, 24 pruned nodes, max_depth=6
[21:27:59] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 68 extra nodes, 26 pruned nodes, max_depth=6
[21:27:59] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 90 extra nodes, 16 pruned nodes, max_depth=6
[21:27:59] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 72 extra nodes, 26 pruned nodes, max_depth=6
[21:27:59] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 84 extra nodes, 16 pruned nodes, max_depth=6
[21:28:00] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 42 extra nodes, 18

[21:28:15] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 82 extra nodes, 28 pruned nodes, max_depth=6
[21:28:15] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 12 extra nodes, 10 pruned nodes, max_depth=6
[21:28:16] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 56 extra nodes, 12 pruned nodes, max_depth=6
[21:28:16] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 44 extra nodes, 42 pruned nodes, max_depth=6
[21:28:16] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 52 extra nodes, 18 pruned nodes, max_depth=6
[21:28:16] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 58 extra nodes, 12 pruned nodes, max_depth=6
[21:28:16] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 40 extra nodes, 34 pruned nodes, max_depth=6
[21:28:17] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 58 extra nodes, 38 pruned nodes, max_depth=6
[21:28:17] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 60 extra nodes, 20 

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 19.2min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 19.2min finished
/home/joaquinfontela/.local/lib/python3.8/site-packages/sklearn/ensemble/_bagging.py:640: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/home/joaquinfontela/.local/lib/python3.8/site-packages/sklearn/ensemble/_bagging.py:644: RuntimeWarning: invalid value encountered in true_divide
  oob_decision_function = (predictions /


BaggingClassifier(base_estimator=XGBClassifier(base_score=None,
                                               booster='gbtree',
                                               colsample_bylevel=None,
                                               colsample_bynode=None,
                                               colsample_bytree=None, gamma=1,
                                               gpu_id=None,
                                               importance_type='gain',
                                               interaction_constraints=None,
                                               label_encoder=False,
                                               learning_rate=0.45,
                                               max_delta_step=None,
                                               max_depth=None,
                                               min_child_weight=None,
                                               missing=nan,
                                               

In [17]:
# xgb_model_2 = XGBClassifier(n_estimators = 350,
#                           subsample = 0.950,
#                           booster = 'gbtree',
#                           gamma = 0.5,
#                           learning_rate = 0.45,
#                           label_encoder = False,
#                           verbosity = 2)
# xgb_model_2.fit(X_train, y_train)

In [18]:
# xgb_model_3 = XGBClassifier(n_estimators = 350,
#                           subsample = 0.750,
#                           booster = 'gbtree',
#                           gamma = 1,
#                           learning_rate = 0.45,
#                           label_encoder = False,
#                           verbosity = 2)
# xgb_model_3.fit(X_train, y_train)

In [19]:
# xgb_model_4 = XGBClassifier(n_estimators = 350,
#                           subsample = 0.80,
#                           booster = 'gbtree',
#                           gamma = 1,
#                           learning_rate = 0.55,
#                           label_encoder = False,
#                           verbosity = 2)
# xgb_model_4.fit(X_train, y_train)

In [20]:
rf_model_1 = RandomForestClassifier(n_estimators = 150,
                                  max_depth = None,
                                  max_features = 45,
                                  min_samples_split = 15,
                                  min_samples_leaf = 1,
                                  criterion = "gini",
                                  verbose=True)
rf_model_1.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:  2.8min finished


RandomForestClassifier(max_features=45, min_samples_split=15, n_estimators=150,
                       verbose=True)

In [21]:
# rf_model_2 = RandomForestClassifier(n_estimators = 250,
#                                   max_depth = None,
#                                   max_features = 45,
#                                   min_samples_split = 15,
#                                   min_samples_leaf = 1,
#                                   criterion = "gini",
#                                   verbose=True,
#                                   n_jobs =-1)
# rf_model_2.fit(X_train, y_train)

In [22]:
import lightgbm as lgb
lgbm_model_1 = lgb.LGBMClassifier(boosting_type='gbdt', 
                         colsample_bytree=1.0,
                         importance_type='split', 
                         learning_rate=0.15,
                         max_depth=None,
                         n_estimators=1600, 
                         n_jobs=-1,
                         objective=None,
                         subsample=1.0, 
                         subsample_for_bin=200000, 
                         subsample_freq=0)

bc_lgbm_model_1 = BaggingClassifier(base_estimator = lgbm_model_1,
                        n_estimators = 30,
                        oob_score = True,
                        bootstrap_features = True,
                        random_state = 0,
                        verbose = 3)


bc_lgbm_model_1.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Building estimator 1 of 30 for this parallel run (total 30)...
Building estimator 2 of 30 for this parallel run (total 30)...
Building estimator 3 of 30 for this parallel run (total 30)...
Building estimator 4 of 30 for this parallel run (total 30)...
Building estimator 5 of 30 for this parallel run (total 30)...
Building estimator 6 of 30 for this parallel run (total 30)...
Building estimator 7 of 30 for this parallel run (total 30)...
Building estimator 8 of 30 for this parallel run (total 30)...
Building estimator 9 of 30 for this parallel run (total 30)...
Building estimator 10 of 30 for this parallel run (total 30)...
Building estimator 11 of 30 for this parallel run (total 30)...
Building estimator 12 of 30 for this parallel run (total 30)...
Building estimator 13 of 30 for this parallel run (total 30)...
Building estimator 14 of 30 for this parallel run (total 30)...
Building estimator 15 of 30 for this parallel run (total 30)...
Building estimator 16 of 30 for this parallel run

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 14.1min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 14.1min finished


BaggingClassifier(base_estimator=LGBMClassifier(learning_rate=0.15,
                                                max_depth=None,
                                                n_estimators=1600),
                  bootstrap_features=True, n_estimators=30, oob_score=True,
                  random_state=0, verbose=3)

In [23]:
# lgbm_model_2 = lgb.LGBMClassifier(boosting_type='gbdt', 
#                          colsample_bytree=1.0,
#                          importance_type='split', 
#                          learning_rate=0.15,
#                          max_depth=25,
#                          n_estimators=1750, 
#                          n_jobs=-1,
#                          objective=None,
#                          subsample=0.7, 
#                          subsample_for_bin=240000, 
#                          subsample_freq=0)
# lgbm_model_2.fit(X_train, y_train)

In [24]:
# lgbm_model_3 = lgb.LGBMClassifier(boosting_type='gbdt', 
#                          colsample_bytree=1.0,
#                          importance_type='split', 
#                          learning_rate=0.20,
#                          max_depth=40,
#                          n_estimators=1450, 
#                          n_jobs=-1,
#                          objective=None,
#                          subsample=0.7, 
#                          subsample_for_bin=160000, 
#                          subsample_freq=0)
# lgbm_model_3.fit(X_train, y_train)

In [25]:
from catboost import CatBoostClassifier
cb_model_1 = CatBoostClassifier(iterations = 7500,
                                eval_metric='Accuracy',
                                loss_function='MultiClass',
                                use_best_model=True,
                                random_seed=1,
                                max_depth = None,
                                boost_from_average = False,
                                verbose=True)
cb_model_1.fit(X_train, y_train, 
          cat_features=['geo_level_2_id', 'geo_level_3_id'],
          eval_set=(X_test,y_test))

Learning rate set to 0.055808
0:	learn: 0.6946008	test: 0.6952318	best: 0.6952318 (0)	total: 253ms	remaining: 31m 34s
1:	learn: 0.6988861	test: 0.6996828	best: 0.6996828 (1)	total: 378ms	remaining: 23m 36s
2:	learn: 0.6984920	test: 0.6992223	best: 0.6996828 (1)	total: 495ms	remaining: 20m 37s
3:	learn: 0.6985750	test: 0.6989665	best: 0.6996828 (1)	total: 600ms	remaining: 18m 45s
4:	learn: 0.6989816	test: 0.6985061	best: 0.6996828 (1)	total: 718ms	remaining: 17m 55s
5:	learn: 0.6997200	test: 0.7000921	best: 0.7000921 (5)	total: 870ms	remaining: 18m 7s
6:	learn: 0.6990853	test: 0.6989154	best: 0.7000921 (5)	total: 1.01s	remaining: 17m 57s
7:	learn: 0.6994130	test: 0.6997851	best: 0.7000921 (5)	total: 1.13s	remaining: 17m 41s
8:	learn: 0.6996743	test: 0.6998874	best: 0.7000921 (5)	total: 1.28s	remaining: 17m 42s
9:	learn: 0.7005248	test: 0.7000921	best: 0.7000921 (5)	total: 1.43s	remaining: 17m 47s
10:	learn: 0.7008525	test: 0.7005014	best: 0.7005014 (10)	total: 1.57s	remaining: 17m 47s
1

92:	learn: 0.7290245	test: 0.7306866	best: 0.7311470 (91)	total: 13s	remaining: 17m 16s
93:	learn: 0.7290577	test: 0.7306354	best: 0.7311470 (91)	total: 13.1s	remaining: 17m 14s
94:	learn: 0.7292319	test: 0.7306354	best: 0.7311470 (91)	total: 13.3s	remaining: 17m 17s
95:	learn: 0.7294310	test: 0.7310447	best: 0.7311470 (91)	total: 13.5s	remaining: 17m 17s
96:	learn: 0.7294933	test: 0.7311982	best: 0.7311982 (96)	total: 13.6s	remaining: 17m 18s
97:	learn: 0.7296841	test: 0.7313005	best: 0.7313005 (97)	total: 13.7s	remaining: 17m 17s
98:	learn: 0.7297505	test: 0.7311982	best: 0.7313005 (97)	total: 13.9s	remaining: 17m 18s
99:	learn: 0.7297961	test: 0.7313005	best: 0.7313005 (97)	total: 14s	remaining: 17m 18s
100:	learn: 0.7299372	test: 0.7313005	best: 0.7313005 (97)	total: 14.2s	remaining: 17m 19s
101:	learn: 0.7300035	test: 0.7313005	best: 0.7313005 (97)	total: 14.3s	remaining: 17m 19s
102:	learn: 0.7300326	test: 0.7311982	best: 0.7313005 (97)	total: 14.5s	remaining: 17m 23s
103:	learn:

182:	learn: 0.7344009	test: 0.7353423	best: 0.7360585 (175)	total: 27s	remaining: 17m 58s
183:	learn: 0.7344963	test: 0.7353934	best: 0.7360585 (175)	total: 27.1s	remaining: 17m 59s
184:	learn: 0.7344838	test: 0.7354446	best: 0.7360585 (175)	total: 27.3s	remaining: 17m 59s
185:	learn: 0.7344631	test: 0.7356492	best: 0.7360585 (175)	total: 27.4s	remaining: 17m 58s
186:	learn: 0.7344631	test: 0.7357516	best: 0.7360585 (175)	total: 27.6s	remaining: 17m 58s
187:	learn: 0.7344838	test: 0.7357004	best: 0.7360585 (175)	total: 27.7s	remaining: 17m 58s
188:	learn: 0.7347327	test: 0.7364678	best: 0.7364678 (188)	total: 27.9s	remaining: 17m 59s
189:	learn: 0.7347410	test: 0.7362632	best: 0.7364678 (188)	total: 28.1s	remaining: 17m 59s
190:	learn: 0.7348572	test: 0.7365190	best: 0.7365190 (190)	total: 28.2s	remaining: 17m 59s
191:	learn: 0.7348862	test: 0.7363655	best: 0.7365190 (190)	total: 28.4s	remaining: 17m 59s
192:	learn: 0.7349526	test: 0.7364167	best: 0.7365190 (190)	total: 28.5s	remaining

273:	learn: 0.7376947	test: 0.7382073	best: 0.7383096 (270)	total: 41.8s	remaining: 18m 22s
274:	learn: 0.7377818	test: 0.7381050	best: 0.7383096 (270)	total: 41.9s	remaining: 18m 22s
275:	learn: 0.7378150	test: 0.7382073	best: 0.7383096 (270)	total: 42.1s	remaining: 18m 22s
276:	learn: 0.7378399	test: 0.7381561	best: 0.7383096 (270)	total: 42.3s	remaining: 18m 21s
277:	learn: 0.7378524	test: 0.7384120	best: 0.7384120 (277)	total: 42.4s	remaining: 18m 21s
278:	learn: 0.7378482	test: 0.7382585	best: 0.7384120 (277)	total: 42.5s	remaining: 18m 21s
279:	learn: 0.7378275	test: 0.7384120	best: 0.7384120 (277)	total: 42.7s	remaining: 18m 21s
280:	learn: 0.7378524	test: 0.7385654	best: 0.7385654 (280)	total: 42.9s	remaining: 18m 22s
281:	learn: 0.7378482	test: 0.7385654	best: 0.7385654 (280)	total: 43.1s	remaining: 18m 22s
282:	learn: 0.7378524	test: 0.7384631	best: 0.7385654 (280)	total: 43.2s	remaining: 18m 22s
283:	learn: 0.7378399	test: 0.7384631	best: 0.7385654 (280)	total: 43.3s	remaini

364:	learn: 0.7394039	test: 0.7401003	best: 0.7402026 (359)	total: 56.5s	remaining: 18m 24s
365:	learn: 0.7394412	test: 0.7400491	best: 0.7402026 (359)	total: 56.7s	remaining: 18m 24s
366:	learn: 0.7394868	test: 0.7401003	best: 0.7402026 (359)	total: 56.8s	remaining: 18m 24s
367:	learn: 0.7394868	test: 0.7401514	best: 0.7402026 (359)	total: 57s	remaining: 18m 24s
368:	learn: 0.7395574	test: 0.7402538	best: 0.7402538 (368)	total: 57.1s	remaining: 18m 23s
369:	learn: 0.7395532	test: 0.7404072	best: 0.7404072 (369)	total: 57.3s	remaining: 18m 23s
370:	learn: 0.7395325	test: 0.7402538	best: 0.7404072 (369)	total: 57.4s	remaining: 18m 23s
371:	learn: 0.7395823	test: 0.7401003	best: 0.7404072 (369)	total: 57.6s	remaining: 18m 23s
372:	learn: 0.7395988	test: 0.7401003	best: 0.7404072 (369)	total: 57.7s	remaining: 18m 23s
373:	learn: 0.7396403	test: 0.7401514	best: 0.7404072 (369)	total: 57.9s	remaining: 18m 23s
374:	learn: 0.7396528	test: 0.7402538	best: 0.7404072 (369)	total: 58.1s	remaining

...

7139:	learn: 0.7791790	test: 0.7491047	best: 0.7498209 (6626)	total: 19m 3s	remaining: 57.7s
7140:	learn: 0.7791915	test: 0.7490024	best: 0.7498209 (6626)	total: 19m 3s	remaining: 57.5s
7141:	learn: 0.7791749	test: 0.7489000	best: 0.7498209 (6626)	total: 19m 3s	remaining: 57.3s
7142:	learn: 0.7791707	test: 0.7488489	best: 0.7498209 (6626)	total: 19m 3s	remaining: 57.2s
7143:	learn: 0.7791956	test: 0.7490024	best: 0.7498209 (6626)	total: 19m 4s	remaining: 57s
7144:	learn: 0.7791998	test: 0.7489512	best: 0.7498209 (6626)	total: 19m 4s	remaining: 56.9s
7145:	learn: 0.7792122	test: 0.7489512	best: 0.7498209 (6626)	total: 19m 4s	remaining: 56.7s
7146:	learn: 0.7792164	test: 0.7489512	best: 0.7498209 (6626)	total: 19m 4s	remaining: 56.5s
7147:	learn: 0.7792247	test: 0.7488489	best: 0.7498209 (6626)	total: 19m 4s	remaining: 56.4s
7148:	learn: 0.7792039	test: 0.7488489	best: 0.7498209 (6626)	total: 19m 4s	remaining: 56.2s
7149:	learn: 0.7792288	test: 0.7488489	best: 0.7498209 (6626)	total: 19m

7228:	learn: 0.7796105	test: 0.7486442	best: 0.7498209 (6626)	total: 19m 17s	remaining: 43.4s
7229:	learn: 0.7796022	test: 0.7486442	best: 0.7498209 (6626)	total: 19m 18s	remaining: 43.2s
7230:	learn: 0.7796436	test: 0.7485931	best: 0.7498209 (6626)	total: 19m 18s	remaining: 43.1s
7231:	learn: 0.7796561	test: 0.7485931	best: 0.7498209 (6626)	total: 19m 18s	remaining: 42.9s
7232:	learn: 0.7796354	test: 0.7485931	best: 0.7498209 (6626)	total: 19m 18s	remaining: 42.8s
7233:	learn: 0.7796395	test: 0.7485419	best: 0.7498209 (6626)	total: 19m 18s	remaining: 42.6s
7234:	learn: 0.7796395	test: 0.7485419	best: 0.7498209 (6626)	total: 19m 18s	remaining: 42.4s
7235:	learn: 0.7796354	test: 0.7485419	best: 0.7498209 (6626)	total: 19m 19s	remaining: 42.3s
7236:	learn: 0.7796644	test: 0.7485419	best: 0.7498209 (6626)	total: 19m 19s	remaining: 42.1s
7237:	learn: 0.7796893	test: 0.7485931	best: 0.7498209 (6626)	total: 19m 19s	remaining: 42s
7238:	learn: 0.7797017	test: 0.7485419	best: 0.7498209 (6626)	

7317:	learn: 0.7799797	test: 0.7490535	best: 0.7498209 (6626)	total: 19m 32s	remaining: 29.2s
7318:	learn: 0.7799216	test: 0.7491047	best: 0.7498209 (6626)	total: 19m 32s	remaining: 29s
7319:	learn: 0.7799216	test: 0.7490024	best: 0.7498209 (6626)	total: 19m 32s	remaining: 28.8s
7320:	learn: 0.7799506	test: 0.7490024	best: 0.7498209 (6626)	total: 19m 32s	remaining: 28.7s
7321:	learn: 0.7799174	test: 0.7489000	best: 0.7498209 (6626)	total: 19m 32s	remaining: 28.5s
7322:	learn: 0.7799299	test: 0.7489000	best: 0.7498209 (6626)	total: 19m 33s	remaining: 28.4s
7323:	learn: 0.7799423	test: 0.7489512	best: 0.7498209 (6626)	total: 19m 33s	remaining: 28.2s
7324:	learn: 0.7799133	test: 0.7487977	best: 0.7498209 (6626)	total: 19m 33s	remaining: 28s
7325:	learn: 0.7798967	test: 0.7487977	best: 0.7498209 (6626)	total: 19m 33s	remaining: 27.9s
7326:	learn: 0.7799340	test: 0.7487977	best: 0.7498209 (6626)	total: 19m 33s	remaining: 27.7s
7327:	learn: 0.7799216	test: 0.7487465	best: 0.7498209 (6626)	to

7406:	learn: 0.7802037	test: 0.7490535	best: 0.7498209 (6626)	total: 19m 46s	remaining: 14.9s
7407:	learn: 0.7802078	test: 0.7491047	best: 0.7498209 (6626)	total: 19m 46s	remaining: 14.7s
7408:	learn: 0.7802286	test: 0.7490535	best: 0.7498209 (6626)	total: 19m 46s	remaining: 14.6s
7409:	learn: 0.7802286	test: 0.7490535	best: 0.7498209 (6626)	total: 19m 47s	remaining: 14.4s
7410:	learn: 0.7802203	test: 0.7492582	best: 0.7498209 (6626)	total: 19m 47s	remaining: 14.3s
7411:	learn: 0.7802244	test: 0.7492582	best: 0.7498209 (6626)	total: 19m 47s	remaining: 14.1s
7412:	learn: 0.7802410	test: 0.7492070	best: 0.7498209 (6626)	total: 19m 47s	remaining: 13.9s
7413:	learn: 0.7802452	test: 0.7492070	best: 0.7498209 (6626)	total: 19m 47s	remaining: 13.8s
7414:	learn: 0.7802452	test: 0.7492070	best: 0.7498209 (6626)	total: 19m 47s	remaining: 13.6s
7415:	learn: 0.7802286	test: 0.7492582	best: 0.7498209 (6626)	total: 19m 48s	remaining: 13.5s
7416:	learn: 0.7802742	test: 0.7493093	best: 0.7498209 (6626

7494:	learn: 0.7805439	test: 0.7495651	best: 0.7498209 (6626)	total: 20m	remaining: 801ms
7495:	learn: 0.7805480	test: 0.7495651	best: 0.7498209 (6626)	total: 20m 1s	remaining: 641ms
7496:	learn: 0.7805687	test: 0.7496163	best: 0.7498209 (6626)	total: 20m 1s	remaining: 481ms
7497:	learn: 0.7805770	test: 0.7495651	best: 0.7498209 (6626)	total: 20m 1s	remaining: 320ms
7498:	learn: 0.7805646	test: 0.7495651	best: 0.7498209 (6626)	total: 20m 1s	remaining: 160ms
7499:	learn: 0.7805605	test: 0.7496163	best: 0.7498209 (6626)	total: 20m 1s	remaining: 0us

bestTest = 0.7498209352
bestIteration = 6626

Shrink model to first 6627 iterations.


In [26]:
# import sklearn as sk
# import sklearn.neural_network
# neuronal_1 = sk.neural_network.MLPClassifier(solver='adam', 
#                                      activation = 'relu',
#                                      learning_rate_init=0.001,
#                                      learning_rate = 'adaptive',
#                                      verbose=True,
#                                      batch_size = 'auto')
# neuronal_1.fit(X_train, y_train)

In [27]:
gb_model_1 = GradientBoostingClassifier(n_estimators = 300,
                                  max_depth = 9,
                                  min_samples_split = 2,
                                  min_samples_leaf = 3,
                                  subsample=0.6,
                                  verbose=True,
                                  learning_rate=0.15)
gb_model_1.fit(X_train, y_train)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           0.8427           0.0681           15.62m
         2           0.7979           0.0412           15.32m
         3           0.7675           0.0291           15.21m
         4           0.7441           0.0216           15.18m
         5           0.7254           0.0164           15.14m
         6           0.7095           0.0134           15.06m
         7           0.6994           0.0101           14.99m
         8           0.6885           0.0085           14.92m
         9           0.6818           0.0069           14.87m
        10           0.6720           0.0068           14.85m
        20           0.6239           0.0023           14.40m
        30           0.5967           0.0014           13.97m
        40           0.5758           0.0011           13.45m
        50           0.5585           0.0008           12.92m
        60           0.5474           0.0003           12.40m
       

GradientBoostingClassifier(learning_rate=0.15, max_depth=9, min_samples_leaf=3,
                           n_estimators=300, subsample=0.6, verbose=True)

In [28]:
eclf_model = EnsembleVoteClassifier(clfs=[bc_xgb_model_1,  #~74.53
                                          rf_model_1, #~73.92
                                          bc_lgbm_model_1, #~74.56
                                          cb_model_1, #~74.92
                                          gb_model_1], #~74.07
                                    weights=[3,1.75,3,6.75,1.875],
                                    fit_base_estimators=False,
                                    voting='soft')

In [29]:
eclf_model.fit(X_train, y_train)

/home/joaquinfontela/.local/lib/python3.8/site-packages/mlxtend/classifier/ensemble_vote.py:166: UserWarning: fit_base_estimators=False enforces use_clones to be `False`
  warnings.warn("fit_base_estimators=False "


EnsembleVoteClassifier(clfs=[BaggingClassifier(base_estimator=XGBClassifier(base_score=None,
                                                                            booster='gbtree',
                                                                            colsample_bylevel=None,
                                                                            colsample_bynode=None,
                                                                            colsample_bytree=None,
                                                                            gamma=1,
                                                                            gpu_id=None,
                                                                            importance_type='gain',
                                                                            interaction_constraints=None,
                                                                            label_encoder=False,
                                      

In [30]:
y_preds = eclf_model.predict(X_test)
f1_score(y_test, y_preds, average='micro')

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:    0.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   46.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   46.5s finished


0.7515604215696307

In [31]:
test_values = pd.read_csv('../../csv/test_values.csv', index_col = "building_id")
test_values

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
300051,17,596,11307,3,20,7,6,t,r,n,...,0,0,0,0,0,0,0,0,0,0
99355,6,141,11987,2,25,13,5,t,r,n,...,1,0,0,0,0,0,0,0,0,0
890251,22,19,10044,2,5,4,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
745817,26,39,633,1,0,19,3,t,r,x,...,0,0,1,0,0,0,0,0,0,0
421793,17,289,7970,3,15,8,7,t,r,q,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310028,4,605,3623,3,70,20,6,t,r,q,...,1,0,0,0,0,0,0,0,0,0
663567,10,1407,11907,3,25,6,7,n,r,n,...,0,0,0,0,0,0,0,0,0,0
1049160,22,1136,7712,1,50,3,3,t,r,n,...,0,0,0,0,0,0,0,0,0,0


In [32]:
test_values_subset = test_values
test_values_subset["geo_level_1_id"] = test_values_subset["geo_level_1_id"].astype("category")
test_values_subset

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
300051,17,596,11307,3,20,7,6,t,r,n,...,0,0,0,0,0,0,0,0,0,0
99355,6,141,11987,2,25,13,5,t,r,n,...,1,0,0,0,0,0,0,0,0,0
890251,22,19,10044,2,5,4,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
745817,26,39,633,1,0,19,3,t,r,x,...,0,0,1,0,0,0,0,0,0,0
421793,17,289,7970,3,15,8,7,t,r,q,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310028,4,605,3623,3,70,20,6,t,r,q,...,1,0,0,0,0,0,0,0,0,0
663567,10,1407,11907,3,25,6,7,n,r,n,...,0,0,0,0,0,0,0,0,0,0
1049160,22,1136,7712,1,50,3,3,t,r,n,...,0,0,0,0,0,0,0,0,0,0


In [33]:
test_values_subset.shape

(86868, 38)

In [34]:
test_values_subset['age_is_leq_than_100'] = (test_values_subset['age'] <= 100).astype(np.int8)
# test_values_subset['age_is_betw_100_and_200'] = ((test_values_subset['age'] > 100) & (test_values_subset['age'] <= 200)).astype(np.int8)
test_values_subset['age_is_greater_than_200'] = (test_values_subset['age'] > 200).astype(np.int8)
test_values_subset[test_values_subset['age'] >= 100]

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,age_is_leq_than_100,age_is_greater_than_200
building_id,,,,,,,,,,,,,,,,,,,,,
3781,27,181,3563,5,100,3,10,t,r,n,...,0,0,0,0,0,0,0,0,1,0
666855,14,1120,3789,2,995,11,4,t,r,n,...,0,0,0,0,0,0,0,0,0,1
823756,15,874,7477,3,100,12,7,t,r,n,...,0,0,0,0,0,0,0,0,1,0
974381,22,1398,3765,2,995,4,5,t,r,n,...,0,0,0,0,0,0,0,0,0,1
118979,7,714,7850,2,100,4,5,t,r,n,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196137,11,131,3126,4,110,11,8,t,r,q,...,0,0,0,0,0,0,0,0,0,0
783720,27,181,2607,2,995,2,6,t,r,n,...,0,0,0,0,0,0,0,0,0,1
880161,13,480,1917,2,995,10,4,t,r,n,...,0,0,0,0,0,0,0,0,0,1


In [35]:
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res) 

features_to_encode = ["geo_level_1_id", "land_surface_condition", "foundation_type", "roof_type",\
                     "position", "ground_floor_type", "other_floor_type",\
                     "plan_configuration", "legal_ownership_status"]
for feature in features_to_encode:
    test_values_subset = encode_and_bind(test_values_subset, feature)
test_values_subset

,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,...,plan_configuration_m,plan_configuration_n,plan_configuration_o,plan_configuration_q,plan_configuration_s,plan_configuration_u,legal_ownership_status_a,legal_ownership_status_r,legal_ownership_status_v,legal_ownership_status_w
building_id,,,,,,,,,,,,,,,,,,,,,
300051,596,11307,3,20,7,6,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
99355,141,11987,2,25,13,5,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
890251,19,10044,2,5,4,5,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
745817,39,633,1,0,19,3,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
421793,289,7970,3,15,8,7,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310028,605,3623,3,70,20,6,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
663567,1407,11907,3,25,6,7,1,1,1,0,...,0,0,0,0,0,0,0,0,1,0
1049160,1136,7712,1,50,3,3,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [36]:
# Genero las predicciones para los test.
preds = eclf_model.predict(test_values_subset)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:    3.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.5min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.5min finished


In [37]:
submission_format = pd.read_csv('../../csv/submission_format.csv', index_col = "building_id")

In [38]:
my_submission = pd.DataFrame(data=preds,
                             columns=submission_format.columns,
                             index=submission_format.index)

In [39]:
my_submission.head()

,damage_grade
building_id,
300051,3
99355,2
890251,2
745817,1
421793,3


In [40]:
my_submission.to_csv('../../csv/predictions/jf/ensemble-vote/jf-model-6-submission.csv')

In [41]:
!head ../../csv/predictions/jf/ensemble-vote/jf-model-6-submission.csv

building_id,damage_grade
300051,3
99355,2
890251,2
745817,1
421793,3
871976,2
691228,1
896100,3
343471,2
